In [1]:
# Install necessary libraries
!pip install datasets transformers rouge-score nltk sacrebleu

# Import libraries and load dataset
from datasets import load_dataset
import torch
import torch.nn as nn
from transformers import GPT2Tokenizer
from sklearn.metrics import f1_score, accuracy_score
from rouge_score import rouge_scorer
import numpy as np

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the ARC-Easy dataset from Hugging Face
dataset = load_dataset("allenai/ai2_arc", "ARC-Easy")
print("Dataset loaded successfully.")

# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2.0 / torch.pi)) * (x + 0.044715 * torch.pow(x, 3))))

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg['emb_dim'], 4 * cfg['emb_dim']),
            GELU(),
            nn.Linear(4 * cfg['emb_dim'], cfg['emb_dim']),
            nn.Dropout(cfg['drop_rate'])
        )

    def forward(self, x):
        return self.layers(x)

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, block_size, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(block_size, block_size), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3)

        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        mask_unsqueezed = mask_bool.unsqueeze(0).unsqueeze(0)
        attn_scores.masked_fill_(mask_unsqueezed, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)

        return context_vec

class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

        self.att = MultiHeadAttention(
            d_in=cfg['emb_dim'],
            d_out=cfg['emb_dim'],
            block_size=cfg['ctx_len'],
            num_heads=cfg['n_heads'],
            dropout=cfg['drop_rate'],
            qkv_bias=cfg['qkv_bias'],
        )

        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg['emb_dim'])
        self.norm2 = LayerNorm(cfg['emb_dim'])
        self.drop_resid = nn.Dropout(cfg['drop_rate'])

    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_resid(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_resid(x)
        x = x + shortcut

        return x

class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()

        self.tok_emb = nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.pos_emb = nn.Embedding(cfg['ctx_len'], cfg['emb_dim'])
        self.drop_emb = nn.Dropout(cfg['drop_rate'])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg['n_layers'])]
        )

        self.final_norm = LayerNorm(cfg['emb_dim'])
        self.out_head = nn.Linear(cfg['emb_dim'], cfg['vocab_size'], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds

        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)

        return logits

# Define the configuration
cfg = {
    'emb_dim': 768,
    'ctx_len': 1024,
    'n_heads': 12,
    'drop_rate': 0.1,
    'qkv_bias': False,
    'vocab_size': 50257,
    'n_layers': 12,
}

# Instantiate the model
model = GPTModel(cfg).cuda()

# Path to the model file in Google Drive
file_path = '/content/drive/MyDrive/model_and_optimizer (1).pth'

# Load the model state
try:
    model_state = torch.load(file_path)
    model.load_state_dict(model_state['model_state_dict'])
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=84070bc91e23650d776696ee04b23abf98475059332d592f7ad6400777b97341
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
   

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2376 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/570 [00:00<?, ? examples/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Model loaded successfully.


In [2]:
def evaluate_arc_challenge(model, dataset, max_samples=100):
    model.eval()  # Set model to evaluation mode
    arc_score = 0
    total = min(len(dataset['test']), max_samples)
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    f1_scores = []
    total_loss = 0

    criterion = nn.CrossEntropyLoss()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    for idx, example in enumerate(dataset['test']):
        if idx >= max_samples:  # Limit number of samples for quick evaluation
            break
        question = example['question']
        options = example['choices']['text']
        answer = example['choices']['label'].index(example['answerKey'])

        option_logits = []
        predictions = []
        for option in options:
            input_text = question + " " + option
            inputs = tokenizer(input_text, return_tensors="pt").to(device)
            with torch.no_grad():
                logits = model(inputs['input_ids'])
                loss = criterion(logits[:, -1, :], torch.tensor([answer], device=device))
                total_loss += loss.item()
            option_logits.append(logits[:, -1, :].mean().item())
            predictions.append(tokenizer.decode(logits[:, -1, :].argmax(dim=-1).cpu()))

        prediction = torch.tensor(option_logits).argmax().item()
        predicted_text = predictions[prediction]

        if prediction == answer:
            arc_score += 1

        rouge = scorer.score(options[answer], predicted_text)
        rouge1_scores.append(rouge['rouge1'].fmeasure)
        rouge2_scores.append(rouge['rouge2'].fmeasure)
        rougeL_scores.append(rouge['rougeL'].fmeasure)
        f1_scores.append(f1_score([answer], [prediction], average='macro'))

    arc_accuracy = arc_score / total
    avg_rouge1 = np.mean(rouge1_scores)
    avg_rouge2 = np.mean(rouge2_scores)
    avg_rougeL = np.mean(rougeL_scores)
    avg_f1 = np.mean(f1_scores)
    avg_loss = total_loss / total

    return {
        "ARC Score": arc_accuracy,
        "ROUGE-1 Score": avg_rouge1,
        "ROUGE-2 Score": avg_rouge2,
        "ROUGE-L Score": avg_rougeL,
        "F1 Score": avg_f1,
        "Average Loss": avg_loss
    }

# Run evaluation
evaluation_results = evaluate_arc_challenge(model, dataset)
for metric, score in evaluation_results.items():
    print(f"{metric}: {score}")


ARC Score: 0.2
ROUGE-1 Score: 0.0
ROUGE-2 Score: 0.0
ROUGE-L Score: 0.0
F1 Score: 0.2
Average Loss: 1933.9406581115722
